# Random Forests & Ensembles

Nesse notebook vamos estudar técnicas para combinar o resultado de múltiplos modelos de forma a construir preditores mais poderosos e mitigar alguns dos problemas que surgem nos métodos que já estudamos

In [ ]:
import matplotlib.pyplot as plt
import seaborn
import pandas as pd
import numpy as np

%matplotlib inline

In [ ]:
%load_ext autoreload
%autoreload 2

import utils

In [ ]:
np.random.seed(42)

plt.rcParams['figure.figsize'] = (8.0, 5.0)

## Relembrando sobre Árvores de Decisão

In [ ]:
x, y, target_names = utils.load_dataset('cancer')

print('instancias X features:', x.shape)

In [ ]:
x.head()

In [ ]:
from sklearn.model_selection import train_test_split

xtrain, xtest, ytrain, ytest = train_test_split(x, y, test_size=100, random_state=42)

In [ ]:
from sklearn.tree import DecisionTreeClassifier

dt = DecisionTreeClassifier(criterion='entropy')
dt.fit(xtrain, ytrain)

In [ ]:
from sklearn.metrics import accuracy_score

ytrain_pred = dt.predict(xtrain)
ytest_pred = dt.predict(xtest)

print('Acurácia no treino:', accuracy_score(ytrain, ytrain_pred))
print('Acurácia no teste:', accuracy_score(ytest, ytest_pred))

In [ ]:
from sklearn.tree import export_graphviz
import graphviz

dot_data = export_graphviz(dt,
                           out_file=None,
                           feature_names=x.columns,
                           class_names=target_names,
                           filled=True, rounded=True, 
                           special_characters=True)
graphviz.Source(dot_data)

Obtivemos uma acurácia no teste de 94%, o que parece bem razoável. Vamos tentar de novo, mas vamos tirar o `random_state=42` do nosso `train_test_split`, o que vai fazer com que os nossos 150 exemplos de teste (e consequentemente os nossos exemplos de treino) sejam sempre diferentes:

In [ ]:
from sklearn.linear_model import LogisticRegression

In [ ]:
xtrain, xtest, ytrain, ytest = train_test_split(x, y, test_size=150)

dt = DecisionTreeClassifier()
dt.fit(xtrain, ytrain)

accuracy_score(ytest, dt.predict(xtest))

## Fronteiras de Decisão

Podemos visualizar as fronteiras de decisão de cada classificador treinado para entender melhor os erros que eles cometem:

In [ ]:
sx = x[['mean concave points', 'worst perimeter']]

xtrain, xtest, ytrain, ytest = train_test_split(sx, y, test_size=150)

dt = DecisionTreeClassifier()
dt.fit(xtrain, ytrain)
    
utils.plot2d(sx.values, y.values, clf=dt)
plt.xlim((-0.01, 0.20))
plt.legend(loc=0)

In [ ]:
lr = LogisticRegression()
lr.fit(xtrain, ytrain)

utils.plot2d(sx.values, y.values, clf=lr)
pp.xlim((-0.01, 0.20))
pp.legend(loc=0)

## Combining (ensembling)

Temos vários classificadores treinados com árvores de decisão. Cada um deles é enviesado de uma forma diferente. Por que não combinar a opinião de todos eles?

In [ ]:
def sample(x, y, f):
    sx = x.sample(frac=f)
    sy = y.loc[sx.index]
    return sx, sy

In [ ]:
xtrain, xtest, ytrain, ytest = train_test_split(x, y, test_size=100)

accs = []
predictions = []
for r in range(100):
    sx, sy = sample(xtrain, ytrain, 0.2)
    ...

predictions = np.array(predictions)

In [ ]:
print(np.mean(accs))

# combine!

A técnica que acabamos de usar é um tipo de **ensembling**, mais especificamente um tipo de **bagging**. Nessa aula vamos aprender outras estratégias e formas mais fáceis de implementá-las.

## Random Forest

É muito simples utilizar random forest no nosso dataset:

In [ ]:
from sklearn.ensemble import RandomForestClassifier

rf = RandomForestClassifier()
rf.fit(xtrain, ytrain)

In [ ]:
ypred = rf.predict(xtest)
accuracy_score(ytest, ypred)

## Exercício: Inadimplência

Nesse exemplo vamos carregar um dataset novo. Cada exemplo é uma pessoa que pediu uma linha de crédito e devemos decidir a probabilidade dessa pessoa nos pagar de volta, ou não. Algumas coisas que vocês devem fazer:

* Liste os atributos do problema.
* Observe se algum atributo é categório (não numérico).
* Observe se algum atributo tem dados faltantes (nulls e nans).
* Observe se o problema é balanceado (o que é isso mesmo?).
* Separe um conjunto de teste e um de treino (quanto pra cada?)
* Treine uma árvore de decisão, uma regressão logística e uma floresta aleatória.
* Utilizem a função `eval_auc` para estimar qual dos modelos acima é melhor.
* Use o parâmetro `n_estimators` de `RandomForestClassifier` para ver como isso impacto do desempenho do seu modelo.
* Plote um gráfico `n_estimators` x `AUC` para sua random forest.
* A partir de qual valor de `n_estimators` você acha que não vale mais a pena aumentar?

In [ ]:
from sklearn.metrics import roc_auc_score

def eval_auc(clf, x, y):
    ypred = clf.predict_proba(x)[:,1]
    return roc_auc_score(y, ypred)

In [ ]:
x, y = utils.load_default()

## Ensembling

Nesse exercício vamos observar problemas de overfitting e underfitting e tentar criar ensembles simples parar mitigar tais problemas. Iremos juntos:

* Plotar os dados e observar a correlação entre x e y
* Utilizar `DecisionTreeRegressor` para treinar e visualizar um regressor nesses dados
* Experimentar com alguns parâmetros diferentes e observar os efeitos nos modelos
* Iterar N vezes para treinar N modelos diferentes com diferentes amostras dos dados
* Combinar os resultados de todos os modelos e observar o resultado

In [ ]:
def sample(x, y, n):
    idx = np.random.randint(len(x), size=n)
    return x[idx], y[idx]

In [ ]:
from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import train_test_split

x, y = utils.load_dataset('regression')

In [ ]:
N = 5
train_sample = 0.5

plt.plot(x, y, '.', label="data")

#...

plt.legend()

## Bagging

Nesse exercício vamos experimentar usar a `BaggingClassifier` do sklearn para criar ensembles com regressão logística e árvores de decisão.

In [ ]:
from sklearn.ensemble import BaggingClassifier

In [ ]:
x, y = utils.load_dataset('default')

xtrain, xtest, ytrain, ytest = train_test_split(x, y, test_size=0.3, stratify=y, random_state=1)

In [ ]:
from sklearn.metrics import roc_auc_score

n_estimators = [1,2,3,4,5,10,15,20]
dt_bag_scores = []
lr_bag_scores = []
for ne in n_estimators:
    dt = DecisionTreeClassifier(max_depth=15, random_state=1)
    lr = LogisticRegression(random_state=1)
    
    dt_bag = BaggingClassifier(dt, n_estimators=ne)
    lr_bag = BaggingClassifier(lr, n_estimators=ne)

    dt_bag.fit(xtrain, ytrain)
    lr_bag.fit(xtrain, ytrain)

    dt_bag_scores.append(eval_auc(dt_bag, xtest, ytest))
    lr_bag_scores.append(eval_auc(lr_bag, xtest, ytest))

    print(ne, dt_bag_scores[-1], lr_bag_scores[-1])


In [ ]:
plt.plot(n_estimators, dt_bag_scores, label='bagging')
plt.plot(n_estimators, lr_bag_scores, label='forest')
plt.legend()

Esse resultado talvez deixe claro por que *random forests* são tão populares e não *random logistic regressions* não :)

## Voting

Vamos usar o `VotingClassifier` do `sklearn` para compor alguns classificadores diferentes.

In [ ]:
x, y = utils.load_dataset('default')

xtrain, xtest, ytrain, ytest = train_test_split(x, y, test_size=0.3, stratify=y, random_state=1)

In [ ]:
from sklearn.ensemble import VotingClassifier
from sklearn.neighbors import KNeighborsClassifier

dt1 = DecisionTreeClassifier(random_state=1)
dt2 = DecisionTreeClassifier(max_depth=12, max_features=5, random_state=42)
lr1 = LogisticRegression()
lr2 = LogisticRegression(C=0.001, penalty='l1')
knn = KNeighborsClassifier()

clfs = [('dt1', dt1),
        ('dt2', dt2),
        ('lr1', lr1),
        ('lr2', lr2),
        ('knn', knn)]

vot = VotingClassifier(clfs, voting='soft')
vot.fit(xtrain, ytrain)

In [ ]:
for name, clf in clfs:
    clf.fit(xtrain, ytrain)
    
    auc = eval_auc(clf, xtest, ytest)
    print(name, auc)
    
print()
print('all', eval_auc(vot, xtest, ytest))

## Mini-competição!

Vamos usar os métodos que aprendemos pra tentar melhorar nosso resultado nesse dataset de inadimplência! Usem a criatividade :)

In [ ]:
x, y = utils.load_dataset('default')

xtrain, xtest, ytrain, ytest = train_test_split(x, y, test_size=0.3, stratify=y, random_state=1)

In [ ]:
clf = DecisionTreeClassifier()
clf.fit(xtrain, ytrain)

eval_auc(clf, xtest, ytest)

## Stacking

Vamos usar os mesmos estimadores base do exemplo anterior, porém ao invés de definir a predição por voto, vamos **treinar outro classificador com as predições dos classificadores base**. Primeiro começamos com nosso split train/test padrão:

In [ ]:
xtrain, xtest, ytrain, ytest = train_test_split(x, y, test_size=0.3, stratify=y, random_state=1)

Agora precisamos fazer um novo split no conjunto de treino para reservar dados para nosso meta-learner.

In [ ]:
xbase, xmeta, ybase, ymeta = train_test_split(xtrain, ytrain, test_size=0.5, random_state=1)

In [ ]:
xmeta_preds = pd.DataFrame(index=ymeta.index)

# ...

In [ ]:
meta_clf = DecisionTreeClassifier(max_depth=7, random_state=1)
# ...

In [ ]:
xtest_meta = pd.DataFrame(index=ytest.index)

# ...

In [ ]:
eval_auc(meta_clf, xtest_meta, ytest)

## De volta ao Kaggle!

![Comp](taxi-competition.png)

Vamos utilizar as técnicas que aprendemos para treinar modelos pra prever quanto tempo uma viagem de taxi em NY vai levar. Depois vamos submeter nossa solução para o Kaggle.

Dessa vez é pra ganhar!!

In [ ]:
df = pd.read_csv('../data/kaggle/train.csv')

In [ ]:
print('Instances x features:', df.shape)
df.head()

Lembrando que já vimos que não precisamos imputar (han??) nenhuma feature:

In [ ]:
df.isnull().sum(axis=0)

Porém precisamos lidar com algumas variáveis não numéricas:

In [ ]:
df.dtypes

Vamos dar uma relembrada rápida na distribuição nosso target (tempo de viagem):

In [ ]:
df.trip_duration.hist(bins=range(0,5000,50))

Agora vamos criar nossas features e nosso target:

In [ ]:
x = df.drop(['trip_duration', 'id', 'pickup_datetime', 'dropoff_datetime', 'store_and_fwd_flag'], axis=1)
y = df['trip_duration']

In [ ]:
xtrain, xtest, ytrain, ytest = train_test_split(x, y, test_size=0.1, random_state=1)

In [ ]:
from sklearn.tree import DecisionTreeRegressor
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor

reg = DecisionTreeRegressor(max_depth=50)
reg.fit(xtrain, ytrain)

In [ ]:
from sklearn.metrics import mean_absolute_error, mean_squared_log_error

def rmsle(ytrue, ypred):
    return np.sqrt(mean_squared_log_error(ytrue, ypred))

In [ ]:
ypred = reg.predict(xtest)

In [ ]:
print(mean_absolute_error(ytest, ypred)/60)
print(rmsle(ytest, ypred))

In [ ]:
print(mean_absolute_error(ytest, ypred)/60)
print(rmsle(ytest, ypred))

### Submission

Para montar nossa submissão precisamos:

* Carregar o arquivo CSV
* Aplicar o mesmo pre-processamento que fizemos no treino (que ainda é bem simples)
* Fazer as predições
* Montar e salvar um arquivo CSV

In [ ]:
df_sub = pd.read_csv('../data/kaggle/test.csv')

In [ ]:
x_sub = df_sub.drop(['id', 'pickup_datetime', 'store_and_fwd_flag'], axis=1)

In [ ]:
y_sub = reg.predict(x_sub)

In [ ]:
sub = pd.DataFrame({'id': df_sub.id, 'trip_duration': y_sub})
sub.to_csv('../data/kaggle/sub1.csv', index=False)